In [1]:
import io
import requests
import pandas as pd
import numpy as np

from scipy import linalg

## Get the data

In [2]:
import requests
import pandas as pd

r = requests.post(
  'https://fink-portal.org/api/v1/sso',
  json={
    'n_or_d': '1465',
    'withEphem': True,
    'output-format': 'json'
  }
)

# Format output in a DataFrame
pdf = pd.read_json(io.BytesIO(r.content))
pdf = pdf.sort_values('Phase')

In [3]:
pdf.head(3)

,index,Date,LAST,HA,Az,H,Dobs,Dhelio,VMag,SDSS:g,...,i:tooflag,i:xpos,i:ypos,d:tracklet,v:classification,v:lastdate,v:firstdate,v:lapse,v:constellation,i:magpsf_red
164,164,2.459175e+06,04:13:01.52,0.201273,186.775076,63.710023,2.542281,3.511448,16.860343,17.303143,...,0,666.5447,734.6332,,Solar System MPC,2020-11-21 07:57:05.996,2020-11-21 07:57:05.996,0.000000,Taurus,12.239638
163,163,2.459177e+06,04:04:21.95,0.083356,182.804538,63.734315,2.541769,3.510093,16.861588,17.304388,...,0,2062.4023,1089.3254,,Solar System MPC,2020-11-23 07:40:35.999,2020-11-23 07:39:14.999,0.000937,Taurus,11.643147
162,162,2.459177e+06,04:38:14.44,0.648257,201.101307,62.255539,2.541770,3.510077,16.861650,17.304450,...,0,2081.8350,1087.7336,,Solar System MPC,2020-11-23 08:14:23.004,2020-11-23 08:14:23.004,0.000000,Taurus,12.210151


## Standard fit

All the bands are aggregated into one lightcurve, and we fit:

$$
\begin{equation}
H = m_{obs} + f(distance) + h(\phi, G_1, G_2) + s(R, \alpha_0, \delta_0)
\end{equation}
$$

for $(H, G_1, G_2, R, \alpha_0, \beta_0)$

In [4]:
from fink_utils.sso.spins import func_hg1g2_with_spin, add_ztf_color_correction, estimate_sso_params

In [5]:
params = ['H', 'G1', 'G2', 'R', 'alpha[deg]', 'delta[deg]']
bounds = (
    [0, 0, 0, 1e-1, 0, -np.pi/2],
    [30, 1, 1, 1, 2*np.pi, np.pi/2]
)

In [6]:
pdf = add_ztf_color_correction(pdf, combined=True)

In [7]:
popt, perr, chisq_red = estimate_sso_params(
    pdf,
    func_hg1g2_with_spin,
    bounds=bounds
)

In [8]:
msg = """
{} = {} +/- {}
{} = {} +/- {}
{} = {} +/- {}
---
{} = {} +/- {}
{} = {} +/- {}
{} = {} +/- {}
--
chi2red = {}
""".format(*np.ravel(list(zip(params, popt, perr))), chisq_red)
print(msg)


H = 11.42121833273937 +/- 0.3445875592694126
G1 = 0.6896769873480129 +/- 0.4464931289578242
G2 = 0.03843879687589978 +/- 0.12248298490039101
---
R = 0.26680189348336286 +/- 0.1319513451385938
alpha[deg] = 2.209387408310816 +/- 0.03398572151000516
delta[deg] = -0.7303241760977177 +/- 0.12501375464433337
--
chi2red = 6.642540126226144



## Generalisation

Before we assumed no wavelength dependency for the $(H, G_1, G_2)$ parameters, and use all data available to fit for 

$$
\begin{equation}
( H, G_1, G_2, R, \alpha_0, \delta_0)
\end{equation}
$$

but in principle we should rather estimate:

$$
\begin{equation}
( H^g, G_1^g, G_2^g, H^r, G_1^r, G_2^r, R, \alpha_0, \delta_0)
\end{equation}
$$

Let's then build a system of equation and solve for:

$$
\begin{align}
H^g &= m_{obs} + f(distance) + h(\alpha, G_1^g, G_2^g) + s(R, \alpha_0, \delta_0) \\
H^r &= m_{obs} + f(distance) + h(\alpha, G_1^r, G_2^r) + s(R, \alpha_0, \delta_0) \\
\end{align}
$$

In [9]:
from fink_utils.sso.spins import estimate_hybrid_sso_params

# Extract relevant information
magpsf_red = pdf['i:magpsf_red'].values
sigmapsf = pdf['i:sigmapsf'].values
phase = np.deg2rad(pdf['Phase'].values)
ra = np.deg2rad(pdf['i:ra'].values)
dec = np.deg2rad(pdf['i:dec'].values)
filters = pdf['i:fid'].values

outdic = estimate_hybrid_sso_params(magpsf_red, sigmapsf, phase, ra, dec, filters)

In [10]:
params = ['H_1', 'G1_1', 'G2_1', 'H_2', 'G1_2', 'G2_2', 'R', 'alpha0', 'delta0']

msg = """
{} = {} +/- {}
{} = {} +/- {}
{} = {} +/- {}
---
{} = {} +/- {}
{} = {} +/- {}
{} = {} +/- {}
--
{} = {} +/- {}
{} = {} +/- {}
{} = {} +/- {}
--
chi2red = {}
""".format(*np.ravel(list(zip(params, [outdic[i] for i in params], [outdic['err'+i] for i in params]))), outdic['chi2red'])
print(msg)


H_1 = 11.649803136095752 +/- 0.3084260488920951
G1_1 = 0.6145410723459498 +/- 0.4005833267797545
G2_1 = 0.13666090209790469 +/- 0.09948396135348006
---
H_2 = 11.29809900171781 +/- 0.529125306103251
G1_2 = 0.9999999999540693 +/- 0.8179948996456996
G2_2 = 0.04236681889044076 +/- 0.17314216400267116
--
R = 0.44894461967946925 +/- 0.05695551126987151
alpha0 = 2.234471753701158 +/- 0.030159451470918947
delta0 = 0.7481148001898047 +/- 0.167662284907787
--
chi2red = 3.2692922297145466



## Notes

The new method is a low level implementation of a least-square instead of the higher curve_fit API. Eventually, for a specific set of params, both least_squares and curve_fit agree (with a small 2% difference on the reduced chi2 though). But they each have pros and cons when it comes to optimisation:

| method | Pros | cons |
|--------|-----|------|
| `least_squares` | Flexible inputs | no errors as input |
| `curve_fit` | account for error estimates as input | no flexibility in the inputs |